In [1]:
%pip install -qq -U diffusers datasets transformers accelerate ftfy pyarrow==9.0.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pathos 0.3.3 requires dill>=0.3.9, but you have dill 0.3.7 which is incompatible.
pathos 0.3.3 requires multiprocess>=0.70.17, but you have multiprocess 0.70.15 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [24]:
import torch
from torchvision import transforms
from PIL import Image
import numpy as np

from tqdm import tqdm
import matplotlib.pyplot as plt

import pandas              as pd
import numpy               as np
import seaborn             as sns

from   PIL     import Image
from   zipfile import ZipFile
import re
import os
import gc
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import lr_scheduler
from torch.distributions import Normal

# Create custom legend
from matplotlib.lines import Line2D
from sklearn.model_selection import train_test_split

In [3]:
from datasets import load_dataset

ds = load_dataset("MatthewWaller/cifar_stable_diffusion")

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

In [4]:
IMAGE_SIZE       = 256
BATCH_SIZE       = 32

NOISE_STEPS      = 500
SUBSET           = 1000

To Train a Model

---

1. Iterate through all images and text
2. Tokenize the text using a prebuilt tokenizer
3. Process the images with an image generator to get all values between 0 to 255 for the three channels
4.


In [5]:
train_data = ds['train']
train_img  = train_data['image'][:SUBSET]
train_txt  = train_data['text'][:SUBSET]

In [6]:
class ImageProcessor:
    def __init__(self, image_size=(224, 224)):
        self.image_size = image_size
        self.transforms = transforms.Compose([
            transforms.Resize(self.image_size),  # Resize image to the target size
            transforms.RandomApply([
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomRotation(degrees=15),
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
                ], p = 0.3
                ),

            transforms.ToTensor(),  # Convert image to tensor with values normalized to [0, 1]
            # transforms.Lambda(lambda x: x * 255)  # Scale tensor values to [0, 255]
        ])

    def process(self, images):
        
        # Apply transformations
        transformed_images = []
        for img in tqdm(images, total = len(images)):
          processed_image = self.transforms(img)
          transformed_images.append(processed_image)
            
        transformed_images = torch.stack(transformed_images)

        return transformed_images


processor   = ImageProcessor(image_size = (IMAGE_SIZE, IMAGE_SIZE))
train_image = processor.process(train_img)

100%|██████████| 1000/1000 [00:04<00:00, 237.65it/s]


In [7]:
from transformers import CLIPTokenizer

# Load the CLIP tokenizer
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")

train_text_tokenized = []
for txt in train_txt:
  # Tokenize text
  tokens = tokenizer(txt, padding=True, truncation=True, return_tensors="pt")
  train_text_tokenized.append(tokens['input_ids'][:,1])


tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

In [8]:
def linear_beta_schedule(timesteps, beta_start=1e-4, beta_end=0.01):
    return torch.linspace(beta_start, beta_end, timesteps)

def compute_alpha(beta):
    alpha = 1.0 - beta
    alpha_cumprod = torch.cumprod(alpha, dim=0)
    return alpha, alpha_cumprod

def add_noise(x0, t, alphas_cumprod, noise=None):
    """
    Add noise to an image x0 at step t.
    Args:
        x0: Original image (batch_size, C, H, W)
        t: Current timestep
        alphas_cumprod: Cumulative product of alphas
        noise: Optional precomputed noise (default: random Gaussian noise)
    Returns:
        xt: Noisy image
    """
    if noise is None:
        noise = torch.randn_like(x0)  # Gaussian noise


    sqrt_alpha_t = torch.sqrt(alphas_cumprod[t])
    sqrt_one_minus_alpha_t = torch.sqrt(1 - alphas_cumprod[t])
    xt = sqrt_alpha_t * x0 + sqrt_one_minus_alpha_t * noise
    return xt


def create_noisy_images(images, plot = False):

    # Define parameters
    timesteps = 100
    beta      = linear_beta_schedule(timesteps, beta_end = 0.02)
    alpha, alphas_cumprod = compute_alpha(beta)

    # Example image (dummy data)
    # x0        = train_image[1]
    dim1, dim2, dim3 = images[0].shape
    final_images = torch.empty(len(images) * timesteps, dim1, dim2, dim3)
    
    for index, img in tqdm(enumerate(images), total = len(images)):
    # Gradually add noise
      noisy_images = []
      for t in range(timesteps):
          xt = add_noise(img, t, alphas_cumprod)
          noisy_images.append(xt)
      
      # Noisiest image first for training
      noisy_images.reverse()
    
      # Convert to tensor for further processing
      noisy_images = torch.stack(noisy_images)  # Shape: (timesteps, batch_size, C, H, W)
    
      # Store images in final_images
      final_images[noisy_images.shape[0] * index:noisy_images.shape[0] * (index + 1), :, : , :] = noisy_images

      if plot:
          # Visualize the noisy images at different timesteps
          fig, axs = plt.subplots(1, 5, figsize=(20, 5))
          for i, t in enumerate([0, 5, 20, 74, 99]):  # Visualize specific timesteps
              axs[i].imshow(noisy_images[t][0].cpu().numpy())
              axs[i].set_title(f"Timestep {t}")
              axs[i].axis("off")

          plt.show()


        
    return final_images


# noisy_images = create_noisy_images(train_image)

In [9]:
import psutil

def get_memory():
    
    memory = psutil.virtual_memory()
    
    available_memory = memory.available
    
    print('Total Available Memory:', available_memory / (1024 ** 3), 'gb')
    

# Training the Stable Diffusion Models


---
1. Left off here trying to get the decoder dimensions to match that of the encoder. 



In [20]:
class ConvEncoder(nn.Module):
    def __init__(self, 
                 input_channels,
                 variational,
                 input_dim,
                 embedding_dim=16, 
                 latent_dim=16,
                 kernel_size = 4,
                 stride   = 2,
                 padding  = 1,
                 output_channels = 128):
        
        super().__init__()
        self.variational     = variational
        self.latent_dim      = latent_dim
        input_dim            = 256
        output_channels      = output_channels
        self.output_channels = output_channels
        
        self.conv_layers = nn.Sequential(
            nn.Conv2d(input_channels, 32, kernel_size=kernel_size, stride=stride, padding=padding),  # Downsample
            nn.ReLU(),
            # Increase Channel size while
            nn.Conv2d(32, 64, kernel_size=kernel_size, stride=stride, padding=padding),  # Downsample again
            nn.ReLU(),
            nn.Conv2d(64, output_channels, kernel_size=kernel_size, stride=stride, padding=padding),  # Downsample again
            nn.ReLU(),
        )
     
        # 3 Layers total
        conv1_dim       = (input_dim - kernel_size + (2*padding))/stride + 1
        conv2_dim       = (conv1_dim - kernel_size + (2*padding))/stride + 1
        self.output_dim = int((conv2_dim - kernel_size + (2*padding))/stride + 1)
        flat_img_dim    = self.output_dim * self.output_dim * output_channels
        
        # Fully connected layers for the latent variables
        self.flatten    = nn.Flatten()
        self.fc_mu      = nn.Linear(flat_img_dim, latent_dim) 
        self.fc_log_var = nn.Linear(flat_img_dim, latent_dim)

    def forward(self, x):
        
        x          = self.conv_layers(x)
        conv_shape = x.shape[1:]
        
        x          = self.flatten(x)
        mu         = self.fc_mu(x)
        log_var    = self.fc_log_var(x)
        conv_shape = x.shape[1:]  # Capture (C, H, W) after conv layers

        if self.variational:
            std = torch.exp(0.5 * log_var)
            q = Normal(mu, std)
            z = q.rsample()  # Reparameterization trick
            return x, mu, log_var, z, conv_shape
        else:
            return mu

class ConvDecoder(nn.Module):
    def __init__(self, output_channels, latent_dim, input_shape, encoder_output_shape):
        super().__init__()
        self.latent_dim           = latent_dim
        self.input_shape          = input_shape
        self.encoder_output_shape = encoder_output_shape
        
        self.fc = nn.Linear(latent_dim, input_shape)
        self.deconv_layers = nn.Sequential(
            nn.ConvTranspose2d(encoder_output_shape[0], 128, kernel_size=4, stride=2, padding=1),  # 32 → 64
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),  # 64 → 128
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),  # 128 → 256
            nn.ReLU(),
            nn.ConvTranspose2d(32, output_channels, kernel_size=3, stride=1, padding=1),  # Refine size to (256, 256)
            nn.Sigmoid(),  # To ensure outputs are in the range [0, 1]
            )

    def forward(self, z):
        x = self.fc(z)
        C, H, W = self.encoder_output_shape
        x = x.view(-1, C, H, W)
        x = self.deconv_layers(x)
        return x
    
    
class ConvVAE(nn.Module):
    def __init__(self, encoder, decoder, variational=True):
        super().__init__()
        self.encoder     = encoder
        self.decoder     = decoder
        self.variational = variational

    def forward(self, x):
        if self.variational:
            _, mu, log_var, z, conv_shape = self.encoder(x)
#             conv_shape = self.encoder.conv_shap
            recon_x = self.decoder(z)
            return recon_x, mu, log_var, z
        else:
            z = self.encoder(x)
            recon_x = self.decoder(z)
            return recon_x, z


def custom_mean_rowwise_rmse(y_true, y_pred):
    """
    Custom metric to calculate the Mean Rowwise Root Mean Squared Error (RMSE).

    Parameters:
    - y_true: The true target values, a PyTorch tensor.
    - y_pred: The predicted values, a PyTorch tensor.

    Returns:
    - Mean Rowwise RMSE as a scalar tensor.
    """

    # Calculate squared difference
    squared_diff = (y_true - y_pred) ** 2

    # Calculate mean squared error for each row
    mse_per_row = torch.mean(squared_diff, dim=1)

    # Calculate root mean squared error for each row
    rmse_per_row = torch.sqrt(mse_per_row)

    # Calculate the mean of RMSE values across all rows
    mean_rmse = torch.mean(rmse_per_row)

    return mean_rmse

# Variational AutoEncoder Code
def loss_function(x, x_recon, mu, log_var, beta = 0.3):
    recon_loss = custom_mean_rowwise_rmse(x, x_recon)
    kl_div     = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

    return recon_loss + beta * kl_div

In [21]:
def build_model(input_data, batch_size, variational = True, stride = 2, kernel_size = 4, padding = 1):
    
  # RGB = 3, Black and white = 1
  channels        = min(input_data[0].shape)
  img_dim         = max(input_data[0].shape)
  encoder         = ConvEncoder(input_channels  = channels, variational = variational, input_dim = img_dim,
                               kernel_size = kernel_size, stride = stride, padding = padding)
    
  
  conv1_dim       = (img_dim - kernel_size + (2*padding))/stride + 1
  conv2_dim       = (conv1_dim - kernel_size + (2*padding))/stride + 1
  conv_output_dim = int((conv2_dim - kernel_size + (2*padding))/stride + 1)

  
#   _, _, _, _, encoder_output_shape = encoder(input_data[:32])
  # Assigned by User 
  output_channels = encoder.output_channels

  encoder_output_shape = (output_channels, conv_output_dim, conv_output_dim)

  # Calculation as a result of filters and strides
  output_dim      = encoder.output_dim
  latent_shape    = output_dim * output_dim * output_channels
  
  decoder         = ConvDecoder(output_channels = channels, 
                                latent_dim = encoder.latent_dim,
                                input_shape = latent_shape,
                                encoder_output_shape = encoder_output_shape)
    
  vae             = ConvVAE(encoder,
                            decoder,
                            variational = variational)

  return vae

BATCH_SIZE = 32
vae = build_model(train_image, batch_size = BATCH_SIZE)

x_train, x_test, x_train_label, x_test_label = train_test_split(train_image, 
                                                                train_text_tokenized,
                                                                random_state = 42, 
                                                                test_size = 0.2)


In [26]:
def train_model(model, x_train, batch_size, variational = True):

    epochs       = 20
    lr           = 0.003
    loss_fn      = loss_function
    optimizer    = torch.optim.Adam(model.parameters(), lr = lr)
    scheduler    = lr_scheduler.ExponentialLR(optimizer, gamma = 0.8)

    batch_size   = batch_size
    batches      = math.floor(x_train.shape[0] / batch_size)
    
    if not isinstance(x_train, torch.Tensor):
        x_train      = torch.tensor(x_train.values, dtype=torch.float32)
        x_validation = torch.tensor(x_validation.values, dtype = torch.float32)
    
    
    losses       = []
    for epoch in tqdm(range(epochs), total = epochs):
        if variational:
            intermediate_losses          = []
            for batch in tqdm(range(1, batches + 1), total = batches):
                # 
                train_batch               = x_train[(batch - 1)*batch_size:batch*batch_size]
              
                # Variational Autoencoder
                x_recon, mu, log_var, z  = model.forward(train_batch)
                
                # AutoEncoder
                loss                     = loss_fn(train_batch, x_recon, mu, log_var)
                
                intermediate_losses.append(loss.detach().numpy())
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                scheduler.step(loss)
                
#                 get_memory()
                
                gc.collect()
                
            intermediate_losses = np.average(intermediate_losses)
            losses.append(intermediate_losses)
            
            print(f'Loss at epoch {epoch + 1}', intermediate_losses)
        
            

    return model


vae_model             = train_model(vae, x_train, BATCH_SIZE)

  4%|▍         | 1/25 [00:02<00:57,  2.40s/it]

Total Available Memory: 57.23537826538086 gb



  8%|▊         | 2/25 [00:04<00:55,  2.41s/it]

Total Available Memory: 57.23627853393555 gb



 12%|█▏        | 3/25 [00:07<00:54,  2.46s/it]

...

Total Available Memory: 57.11015701293945 gb



 96%|█████████▌| 24/25 [01:02<00:02,  2.53s/it]

Total Available Memory: 57.103336334228516 gb



100%|██████████| 20/20 [20:47<00:00, 62.38s/it]

Total Available Memory: 57.097782135009766 gb
Loss at epoch 20 0.2186731


In [47]:
reconstructed_images = vae(x_test[:32])
reconstructed_images = torch.round(reconstructed_images[0] * 255)
reconstructed_image  = reconstructed_images[0].permute(1,2, 0).detach().numpy()
reconstructed_image  = reconstructed_image.astype(int)